In [141]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from category_encoders import TargetEncoder
import seaborn as sns
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from sklearn.model_selection import train_test_split
from category_encoders import CatBoostEncoder
import numpy as np

In [142]:
train=pd.read_csv('/kaggle/input/playground-series-s4e1/train.csv')
test=pd.read_csv('/kaggle/input/playground-series-s4e1/test.csv')

In [143]:
train.nunique()

id                 165034
CustomerId          23221
Surname              2797
CreditScore           457
Geography               3
Gender                  2
Age                    71
Tenure                 11
Balance             30075
NumOfProducts           4
HasCrCard               2
IsActiveMember          2
EstimatedSalary     55298
Exited                  2
dtype: int64

In [144]:
train['CustomerId'].value_counts()

CustomerId
15682355    121
15570194     99
15585835     98
15595588     91
15793331     90
           ... 
15602984      1
15629846      1
15632438      1
15765263      1
15754536      1
Name: count, Length: 23221, dtype: int64

In [145]:
duplicates = train.duplicated(subset=['CustomerId'], keep=False)

# Select rows that are duplicates
duplicate_rows = train[duplicates]

In [146]:
duplicate_rows[duplicate_rows['CustomerId'] == 15674932]

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
671,671,15674932,Genovesi,619,France,Female,37.0,8,0.00,2,1.0,0.0,162858.29,1
4134,4134,15674932,K'ung,667,France,Female,43.0,9,0.00,2,1.0,1.0,119882.70,0
4768,4768,15674932,Williamson,658,Spain,Female,43.0,6,0.00,1,1.0,0.0,16301.91,1
19161,19161,15674932,Norman,670,Spain,Female,50.0,7,0.00,1,1.0,0.0,177528.92,1
19466,19466,15674932,Ma,575,France,Male,42.0,1,0.00,1,0.0,0.0,113581.85,1
25069,25069,15674932,Cameron,757,Spain,Male,30.0,9,0.00,2,1.0,1.0,177528.92,0
25768,25768,15674932,Page,456,France,Female,40.0,4,0.00,2,1.0,1.0,167336.78,0
27170,27170,15674932,Hsia,656,France,Male,39.0,5,0.00,2,0.0,0.0,126013.58,0
31877,31877,15674932,Cameron,602,Spain,Female,34.0,7,0.00,2,1.0,0.0,145937.99,0


In [147]:
has_duplicates = train.duplicated().any()

if has_duplicates:
    print("The DataFrame contains duplicates.")
else:
    print("The DataFrame does not contain duplicates.")

The DataFrame does not contain duplicates.


In [148]:
train_set=train.drop('Exited',axis=1 )

In [149]:
y=train['Exited']
y

0         0
1         0
2         0
3         0
4         0
         ..
165029    0
165030    0
165031    0
165032    0
165033    1
Name: Exited, Length: 165034, dtype: int64

In [150]:
id=test['id']
id

0         165034
1         165035
2         165036
3         165037
4         165038
           ...  
110018    275052
110019    275053
110020    275054
110021    275055
110022    275056
Name: id, Length: 110023, dtype: int64

In [151]:
train_set.drop(['id'],axis=1,inplace=True)
test.drop(['id',],axis=1,inplace=True)

# log transform

In [152]:
# Assuming df is your DataFrame containing numerical features
numerical_features = train_set.select_dtypes(exclude=['object']).columns
#non_obj_column=df.select_dtypes(exclude=['object'])

# Calculate skewness for numerical features
skewed_features = train_set[numerical_features].apply(lambda x: x.skew()).sort_values(ascending=False)

# Select features with skewness greater than a threshold (e.g., 0.5)
skewed_threshold = 0.75
skewed_features = skewed_features[abs(skewed_features) > skewed_threshold]

# Apply log transformation to skewed numerical features
for feature in skewed_features.index:
    train_set[feature] = np.log1p(train_set[feature])  # Log transformation to handle skewness
    test[feature]=np.log1p(test[feature])

# encoding with target  or catBoost encoding

### cat encoding

In [153]:
categorical_columns = train_set.select_dtypes(include=['object']).columns
categorical_columns

Index(['Surname', 'Geography', 'Gender'], dtype='object')

In [154]:

#categorical_cols = ['categorical_feature1', 'categorical_feature2', ...]  # Replace with your categorical columns

# Perform CatBoost encoding
encoder = CatBoostEncoder(cols=categorical_columns)
train_set_encoded = encoder.fit_transform(train_set, y)
test_set_encoded = encoder.transform(test)

## target encooding

In [135]:
# categorical_columns = train_set.select_dtypes(include=['object']).columns
# mode_values = train_set[categorical_columns].mode().iloc[0]
# encoder = TargetEncoder(cols=categorical_columns, handle_unknown='value')
# train_set_encoded = encoder.fit_transform(train_set, y)
# test_set_encoded = encoder.transform(test)

In [136]:

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
def pca_finc(df,percintage):


    # Assuming df is your DataFrame with numerical columns
    # Extract the features (X) from the DataFrame
    df= df.select_dtypes(include=['object'])
    X = df.values

    # Step 1: Standardize the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Step 2: Perform PCA using scikit-learn
    pca = PCA(n_components=percintage)  # You can set the number of components or explained variance
    X_pca = pca.fit_transform(X_scaled)

    # Create a new DataFrame with the principal components
    columns_pca = [f'PC{i+1}' for i in range(X_pca.shape[1])]
    df_pca = pd.DataFrame(data=X_pca, columns=columns_pca)

    # Concatenate the new DataFrame with the original DataFrame if needed
    df_combined = pd.concat([df, df_pca], axis=1)

    # Display the resulting DataFrame
    print(df_pca.shape)
    return df_pca

In [137]:
#data_pca=pca_finc(train_set,.85)

In [138]:
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

# Define class weights
#class_weights = {0: 1, 1: 2}  class_weights=class_weights,

# Initialize CatBoostClassifier with class_weights
model = CatBoostClassifier(verbose=False,random_state=42,task_type='GPU')

# Define k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform k-fold cross-validation with ROC AUC scoring
cv_scores_roc_auc = cross_val_score(model, train_set_encoded, y, cv=kfold, scoring='roc_auc')

# Print cross-validation scores
print("Cross-validation ROC AUC scores:", cv_scores_roc_auc)
print("Mean ROC AUC score:", np.mean(cv_scores_roc_auc))

Cross-validation ROC AUC scores: [0.89121238 0.88736034 0.88738615 0.89163318 0.8889879 ]
Mean ROC AUC score: 0.8893159879098231


In [77]:
model.fit(train_set_encoded,y)

In [81]:
test_probabilities = model.predict_proba(test_set_encoded)[:,1]

# Create a DataFrame with IDs and predicted probabilities

submission_df = pd.DataFrame({'id': id, 'Exited': test_probabilities})

# Save the DataFrame to a CSV file
submission_df.to_csv('submission.csv', index=False)

In [ ]:
/kaggle/working/submission1.csv

In [82]:
submission_df

,id,Exited
0,165034,0.019616
1,165035,0.851621
2,165036,0.023444
3,165037,0.214780
4,165038,0.371841
...,...,...
110018,275052,0.033990
110019,275053,0.131052
110020,275054,0.017734
110021,275055,0.176758


In [1]:
# from lightgbm import LGBMClassifier
# from sklearn.model_selection import cross_val_score, KFold
# import numpy as np

# # Initialize LGBMClassifier
# model = LGBMClassifier(n_estimators=500, verbose=-1)  # You can adjust other hyperparameters as needed

# # Define k-fold cross-validation
# kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# # Perform k-fold cross-validation with ROC AUC scoring
# cv_scores_roc_auc = cross_val_score(model, train_set_encoded, y, cv=kfold, scoring='roc_auc', n_jobs=-1)

# # Print cross-validation scores
# print("Cross-validation ROC AUC scores:", cv_scores_roc_auc)
# print("Mean ROC AUC score:", np.mean(cv_scores_roc_auc))

In [155]:
# from xgboost import XGBClassifier
# from sklearn.model_selection import cross_val_score, KFold
# import numpy as np

In [156]:
# # Initialize XGBClassifier
# model = XGBClassifier(verbosity=0)  # You can adjust other hyperparameters as needed

# # Define k-fold cross-validation
# kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# # Perform k-fold cross-validation with ROC AUC scoring
# cv_scores_roc_auc = cross_val_score(model, train_set_encoded, y, cv=kfold, scoring='roc_auc')

# # Print cross-validation scores
# print("Cross-validation ROC AUC scores:", cv_scores_roc_auc)
# print("Mean ROC AUC score:", np.mean(cv_scores_roc_auc))

Cross-validation ROC AUC scores: [0.89096211 0.886067   0.88558092 0.88924953 0.88843005]
Mean ROC AUC score: 0.8880579223036594


AttributeError: 'Dataset' object has no attribute 'head'

# submition

In [51]:
submission_df

,id,Exited
0,165034,0.019678
1,165035,0.850877
2,165036,0.023252
3,165037,0.213924
4,165038,0.368083
...,...,...
110018,275052,0.033943
110019,275053,0.117097
110020,275054,0.017953
110021,275055,0.174983
